## Fectching Vector Data from GDI using SDK

Below script runs the command line to fetch vector data using GDI SDK
1. The client credentials (id and secret) are made avialable for download at the registering as user of GDI platform. In case credentials are lost, go to your profile, reset the credentials and download for further use.
2. For each data of interest, 'dataset_id' can be copied from the catalogue page
3. In case file_name is specified without file extension (.geojson) the code will automatically add file extension and save in 'Data' folder



In [ ]:
'''
Install gdi python SDK using below pip statement
pip install git+https://github.com/datakaveri/gdi-python-sdk.git
-------------------------------------------------------------------------------
Command line:
gdi get-vector-data --client-id <client_id> --client-secret <client_secret> --role consumer --resource-id <dataset_id> --store-artifact local --config-path 'config.json' --file-path <file_name>
'''

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
import os
import time

# ----------------------------------------
# Create interactive widgets for user input
# ----------------------------------------

# Widget for client ID (text input)
client_id_widget = widgets.Text(
    description="Client ID:",
    placeholder="Enter Client ID",
    layout=widgets.Layout(width='90%')
)

# Widget for client secret (hidden input)
client_secret_widget = widgets.Password(
    description="Client Secret:",
    placeholder="Enter Client Secret",
    layout=widgets.Layout(width='90%')
)

# Widget for resource ID (text input)
resource_id_widget = widgets.Text(
    description="Resource ID:",
    placeholder="Enter Resource ID",
    layout=widgets.Layout(width='90%')
)

# Widget for output filename (text input)
filename_widget = widgets.Text(
    description="Filename:",
    placeholder="e.g. Road_Varanasi",
    layout=widgets.Layout(width='90%')
)

# Button to execute the command
execute_button = widgets.Button(
    description="Fetch Resource",
    button_style="success"   # Makes the button green
)

# Output area to display logs and messages
output = widgets.Output()

# ------------------------------------------------
# Debounce logic to prevent multiple rapid clicks
# ------------------------------------------------

last_click_time = 0

def on_click(b):
    """
    Handler function triggered when the button is clicked.
    It reads the widget values and runs the GDI SDK command.
    """
    global last_click_time

    # Debounce: ignore clicks occurring within 0.5 seconds
    current_time = time.time()
    if current_time - last_click_time < 0.5:
        return
    last_click_time = current_time

    with output:
        clear_output()

        # Read user inputs from widgets
        client_id = client_id_widget.value.strip()
        client_secret = client_secret_widget.value.strip()
        resource_id = resource_id_widget.value.strip()
        filename = filename_widget.value.strip()

        # Ensure the output filename has a .geojson extension
        if not filename.endswith(".geojson"):
            filename += ".geojson"

        # Create output folder called 'Data' if it doesn't exist
        output_dir = "Data"
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, filename)

        # Prepare the GDI SDK command to fetch vector data
        cmd = [
            "gdi", "get-vector-data",
            "--client-id", client_id,
            "--client-secret", client_secret,
            "--role", "consumer",
            "--resource-id", resource_id,
            "--store-artifact", "local",
            "--config-path", "config.json",
            "--file-path", output_path
        ]

        # Print the command for debugging purposes
        print("Executing command:")
        print(" ".join(cmd))
        print()

        try:
            # Run the command and capture output
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)

            # Print success message and output file path
            print("[DONE] Collection download complete.")
            print(f"GDI vector resource successfully downloaded to:\n{output_path}")

        except subprocess.CalledProcessError as e:
            # Print error message if command fails
            print("[ERROR] Failed to fetch resource.")
            print(e.stderr)

# ------------------------------------------------
# Attach the click handler to the button
# ------------------------------------------------

execute_button.on_click(on_click)

# ------------------------------------------------
# Display all widgets and the output area
# ------------------------------------------------

display(
    client_id_widget,
    client_secret_widget,
    resource_id_widget,
    filename_widget,
    execute_button,
    output
)


Text(value='', description='Client ID:', layout=Layout(width='90%'), placeholder='Enter Client ID')

Password(description='Client Secret:', layout=Layout(width='90%'), placeholder='Enter Client Secret')

Text(value='', description='Resource ID:', layout=Layout(width='90%'), placeholder='Enter Resource ID')

Text(value='', description='Filename:', layout=Layout(width='90%'), placeholder='e.g. Road_Varanasi')

Button(button_style='success', description='Fetch Resource', style=ButtonStyle())

Output()

In [4]:
import os 
import geopandas as gpd
input_dir = "./Data"
vector_data = "road.geojson" #replace with your filename
filepath = os.path.join(input_dir, vector_data)
gdf = gpd.read_file(filepath)

# Check basic structure
print("----------------------------------------------------------------------------------------")
print("DATA EXPLORATION")
print("----------------------------------------------------------------------------------------")
print("First few rows:")
print(gdf.head())
print("----------------------------------------------------------------------------------------")
print("CRS:", gdf.crs)
print("Number of features:", len(gdf))

# Check geometry types and nulls
print("Geometry types:", gdf.geometry.geom_type.unique())
print("Any null geometries:", gdf.geometry.isna().any())
print("Sample coordinates:", gdf.geometry.iloc[0])


----------------------------------------------------------------------------------------
DATA EXPLORATION
----------------------------------------------------------------------------------------
First few rows:
   CATEGORY SUBCAT  NAME STATUS PAVEMENT SRFC_TYP CRRG_WAY BULT_ON DRECTN  \
0         1      6  None      1        2        2        1       1   None   
1         1      6  None      1        2        2        1       1   None   
2         1      6  None      1        2        2        1       1   None   
3         1      6  None      1        2        2        1       1   None   
4         1      6  None      1        2        2        1       1   None   

  O_DRECTN ADL_INFO                                           geometry  
0     None     None  MULTILINESTRING ((82.72979 25.51358, 82.72973 ...  
1     None     None  MULTILINESTRING ((82.71749 25.51475, 82.71761 ...  
2     None     None  MULTILINESTRING ((82.73286 25.52538, 82.73268 ...  
3     None     None  MULTILINESTRI